In [1]:
import torch
import numpy as np
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
M_t = 40
N_t = 20

In [3]:
M_a = 20
N_a = 3

In [4]:
def x_t(i):
    return i * 0.5

def y_t(j):
    return j * 0.5

def x_a(i):
    return i * 10 - 5

def y_a(j):
    return j * 10 - 15

In [48]:
def I_s(x, y, x_o, y_o, I_0, sigma_I):
    return I_0 * np.exp( - ((x - x_o)**2 + (y - y_o)**2) / (2 * (sigma_I**2)))

def I_t(x, y, x_o=10, y_o=5, I_0=2.5, sigma_I=0.3):
    return I_s(x, y, x_o, y_o, I_0, sigma_I)

def I_a(x, y, x_o=100, y_o=15, I_0=3.6, sigma_I=0.3):
    return I_s(x, y, x_o, y_o, I_0, sigma_I)

In [49]:
def phi_s(x, y, x_i, y_j, phi_0, sigma_phi):
    return phi_0 * np.exp( - ((x - x_i)**2 + (y - y_j)**2) / (2 * (sigma_phi**2)))

def phi_t(x, y, x_i, y_j, phi_0=1, sigma_phi=1):
    return phi_s(x, y, x_i, y_j, phi_0, sigma_phi)

def phi_a(x, y, x_i, y_j, phi_0=1, sigma_phi=10):
    return phi_s(x, y, x_i, y_j, phi_0, sigma_phi)

In [5]:
def L_s(D_x, D_y, L_ex, sigma_ex, L_in, sigma_in):
    ve = L_ex * np.exp( - (D_x**2 + D_y**2) / (2 * (sigma_ex**2)))
    vi = L_in * np.exp( - (D_x**2 + D_y**2) / (2 * (sigma_in**2)))
    
    return ve - vi

def L_t(i, j, h, k, L_ex_t=0.15, sigma_ex_t=1, L_in_t=0.05, sigma_in_t=1):
    L_ex_t = 0.15
    sigma_ex_t = 1
    
    L_in_t = 0.05
    sigma_in_t = 1
    
    D_x = x_t(h + 1) - x_t(i + 1)
    D_y = y_t(k + 1) - y_t(j + 1)
    
    return L_s(D_x, D_y, L_ex_t, sigma_ex_t, L_in_t, sigma_in_t)

def L_a(D_x, D_y, L_ex_a=0.15, sigma_ex_a=20, L_in_a=0.05, sigma_in_a=80):
    L_ex_a = 0.15
    sigma_ex_a = 20
    
    L_in_a = 0.05
    sigma_in_a = 80

    return L_s(D_x, D_y, L_ex_a, sigma_ex_a, L_in_a, sigma_in_a)

In [38]:
def L2_t(i, j):
    conn_t = torch.zeros(N_t, M_t, device=device)
    
    for h in range(M_t):
        for k in range(N_t):
            if h + 1 == i and k + 1 == j:
                conn_t[k][h] = 0
            else:
                conn_t[k][h] = L_t(i, j, h, k)
    
    return conn_t

In [40]:
L_hk_t(1, 1).shape

torch.Size([20, 40])

In [6]:
fmin_t = -0.12
fmax_t = 1
qc_t = 19.43
r_t = 0.34

In [7]:
fmin_a = -0.12
fmax_a = 1
qc_a = 19.43
r_a = 0.34

In [8]:
fmin_m = 0
fmax_m = 1
qc_m = 12
r_m = 0.6

In [9]:
tau = 20

In [31]:
area_t = torch.zeros(N_t, M_t, device=device)

In [37]:
area_t.shape

torch.Size([20, 40])

In [33]:
def G_m(q_m):
    factor = np.exp((q_m - qc_m) * r_t)
    return (fmin_m + fmax_m * factor) / (1 + factor)
    
def z_m(q_m):
    F_val = G_m(q_m)
    if F_val >= 0:
        return F_val
    else:
        return 0.0

In [65]:
def q_t(area, i, j):
    return area[j][i]

def F_t(area, i, j):
    factor = np.exp((q_t(area, i, j) - qc_t) * r_t)
    return (fmin_t + fmax_t * factor) / (1 + factor)
    
def z_t(area, i, j):
    F_val = F_t(area, i, j)
    if F_val >= 0:
        return F_val
    else:
        return 0.0

In [41]:
# def q_t(area):
#     return area

# def F_t(area):
#     factor = torch.exp((area - qc_t) * r_t)
#     return (fmin_t + fmax_t * factor) / (1 + factor)
    
# def z_t(area):
#     cond = F_t(area)
#     t1 = F_t(area)
#     t2 = torch.zeros(area.shape[0], area.shape[1], device=device)
#     return torch.where(cond >= 0, t1, t2)

In [62]:
def I2_t(area, i, j):    
    return L_hk_t(i, j) * z_t(area, i, j)

def phi2_t(area, d=0.5, delta_x_l=0.2, delta_y_n=0.2):
    area_t = torch.zeros(area.shape[0], area.shape[1], device=device)
    
    for i in range(M_t):
        for j in range(N_t):
            sum_phi_t = 0.0
            
            for x_l in np.arange(delta_x_l, 20, delta_x_l):
                for y_n in np.arange(delta_y_n, 10, delta_y_n):
                    x_i = d * (i + 1)
                    y_j = d * (j + 1)
                    
                    sum_phi_t += I_t(x_l, y_n) * phi_t(x_l, y_n, x_i, y_j)
            
            area_t[j][i] = sum_phi_t
                    
    return area_t

In [63]:
I2_t(area_t, 1, 1).shape

torch.Size([20, 40])

In [ ]:
phi2_t(area_t).shape

In [ ]:
def B2_t(q_m):
    return B_t() + z_m(q_m)

def u_t(area, i, j, q_m):
    return phi2_t(i, j) + I2_t(area, i, j) + B2_t(q_m)

def fire_t(area, q_m):
    for i in range(M_t):
        for j in range(N_t):
            area[j][i] = -q_t(area, i, j) + u_t(area, i, j, q_m)

In [ ]:
def q_a(area, i, j):
    return area[j][i]

def F_a(area, i, j):
    factor = np.exp((q_a(area, i, j) - qc_a) * r_a)
    return (fmin_a + fmax_a * factor) / (1 + factor)
    
def z_a(area, i, j):
    F_val = F_t(area, i, j)
    if F_val >= 0:
        return F_val
    else:
        return 0.0

In [ ]:
def I2_a(area, i, j):
    sum_val = 0.0
    for k in range(M_a):
        for h in range(N_a):
            D_x = x_a(i + 1) - x_a(k + 1)
            D_y = y_a(j + 1) - y_a(h + 1)
            sum_val += L_a(D_x, D_y) * z_a(area, i, j)
    return sum_val

def phi2_a(i, j, d=0.5, delta_x_l=0.2, delta_y_n=0.2):
    sum_phi_a = 0.0
    
    for x_l in np.arange(delta_x_l, 200, delta_x_l):
        for y_n in np.arange(delta_y_n, 30, delta_y_n):
            x_i = d * (i + 1) - 5
            y_j = d * (j + 1) - 15
                    
            sum_phi_a += I_a(x_l, y_n) * phi_a(x_l, y_n, x_i, y_j)
            
    return sum_phi_t

In [ ]:
def B2_a(q_m):
    return B_a() + z_m(q_m)

def u_a(area, i, j, q_m):
    return phi2_a(i, j) + I2_a(area, i, j) + B2_a(q_m)

def fire_a(area, q_m):
    for i in range(M_t):
        for j in range(N_t):
            area[j][i] = -q_a(area, i, j) + u_a(area, i, j, q_m)

In [ ]:
def u_m(area):
    sum_t = 0
    for i in range(M_t):
        for j in range(N_t):
            sum_t += W_t() * z_t(area, i, j)
            
    sum_a = 0
    for i in range(M_a):
        for j in range(N_a):
            sum_a += W_a(D_a(i, j)) * z_a(area, i, j)
    
    return sum_t + sum_a

In [ ]:
q_m = 0.0

area_t = np.zeros((N_t, M_t))

for i in tqdm(range(int(200 / 0.4))[:10]):
    q_m = -q_m + u_m(area_t)
    fire_t(area_t, q_m)